<a href="https://colab.research.google.com/github/sophielu05/114-1-/blob/main/HW6_%E8%AA%B2%E8%A1%A8%E6%9F%A5%E8%A9%A2%E8%88%87%E6%8F%90%E9%86%92%E7%B4%A0%E6%9D%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
pip install gspread pandas gradio

In [41]:
pip install pandas requests gradio

In [42]:
import gradio as gr
import pandas as pd
import requests
from typing import Tuple, List
from io import StringIO

# --- 配置區 ---
# 您的 Google Sheet ID
SPREADSHEET_ID = 'https://docs.google.com/spreadsheets/d/1NkL_6hWQZtNkLjTO-YBGgEewK6p0kynV0RFgtVXgF3U/edit?gid=999624801#gid=999624801'
# 課表資料所在分頁的 GID (網址列 gid= 後面的數字，請確認此 GID 999624801 是正確的分頁)
SHEET_GID = '999624801'
# --- 配置區結束 ---

# 預先定義所有星期的選項
DAY_OPTIONS = ["星期一", "星期二", "星期三", "星期四", "星期五"]
SCHEDULE_DF = None

def load_schedule_data(sheet_id: str, gid: str) -> pd.DataFrame:
    """
    透過公開 CSV 網址讀取 Google 試算表資料。
    """
    # 這是公開讀取 URL 格式
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}'

    try:
        # 使用 requests 取得資料
        response = requests.get(url, timeout=10)
        response.encoding = 'utf-8' # 確保中文編碼正確
        response.raise_for_status() # 對錯誤的 HTTP 狀態碼拋出異常

        csv_data = StringIO(response.text)

        # 讀取 CSV 資料，假設第一列是標題
        df = pd.read_csv(csv_data)

        # 進行基本的資料清理和標準化
        df.columns = [col.strip() for col in df.columns]

        # Rename '教室' to '地點'
        if '教室' in df.columns:
            df = df.rename(columns={'教室': '地點'})
        else:
            print("警告：試算表中未找到 '教室' 欄位，無法自動重命名為 '地點'。")

        # Handle '上課時間' to '星期' conversion
        if '上課時間' in df.columns:
            # Create a new '星期' column by extracting the day part from '上課時間'
            # Assuming '上課時間' starts with "星期X"
            df['星期'] = df['上課時間'].astype(str).str[:3] # Extract "星期X"
        else:
            print("警告：試算表中未找到 '上課時間' 欄位，無法自動生成 '星期' 欄位。")

        # 檢查關鍵欄位：'星期', '課程名稱', '地點'
        required_cols = ['星期', '課程名稱', '地點']
        if not all(col in df.columns for col in required_cols):
            missing_cols = [col for col in required_cols if col not in df.columns]
            print(f"🚨 錯誤：在嘗試重命名後，仍缺少關鍵欄位：{missing_cols}。目前偵測到的欄位: {list(df.columns)}")
            raise ValueError(f"試算表標題錯誤")

        # 確保 '攜帶/閱讀事項' 欄位存在
        if '攜帶/閱讀事項' not in df.columns:
             df['攜帶/閱讀事項'] = ''

        # 存入全域變數
        global SCHEDULE_DF
        SCHEDULE_DF = df

        return df

    except requests.exceptions.HTTPError as e:
        # 🚨 最常見的錯誤：400 (Bad Request) 或 404 (Not Found)
        print(f"🚨 HTTP 錯誤 (請檢查 Sheet ID/GID 或分享設定): {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"🚨 讀取試算表時發生錯誤: {e}")
        return pd.DataFrame()

def get_schedule_and_summary(day_to_search: str) -> Tuple[str, str]:
    """
    根據選定的星期，過濾當日課表，並總結本週提醒事項。
    """
    global SCHEDULE_DF

    # 確保資料已載入
    if SCHEDULE_DF is None or SCHEDULE_DF.empty:
        if SCHEDULE_DF is None:
            load_schedule_data(SPREADSHEET_ID, SHEET_GID)

        if SCHEDULE_DF is None or SCHEDULE_DF.empty:
             # Gradio 介面輸出錯誤訊息
             return "⚠️ 資料載入失敗，請檢查 Sheet ID、GID 是否正確，並確認試算表已設為公開。", ""

    df = SCHEDULE_DF

    # --- 1. 輸出當日課程、地點 ---
    df['星期'] = df['星期'].astype(str) # 確保比對類型一致
    daily_courses = df[df['星期'] == day_to_search]
    daily_schedule_list = []

    if not daily_courses.empty:
        for index, row in daily_courses.iterrows():
            course = row['課程名稱']
            location = row['地點']
            daily_schedule_list.append(f"• 課程：{course} / 地點：{location}")

        daily_schedule_text = f"【{day_to_search} 課表】\n" + "\n".join(daily_schedule_list)
    else:
        daily_schedule_text = f"【{day_to_search} 課表】\n本日無排定課程。"

    # --- 2. 彙整本週行前提醒 (AI 點子) ---
    all_reminders_series = df['攜帶/閱讀事項'].dropna().astype(str).str.strip()

    unique_reminders = set(all_reminders_series[all_reminders_series != ''])

    weekly_reminder_text = ""
    if unique_reminders:
        reminders_list = list(unique_reminders)
        weekly_reminder_text = "【本週重點行前提醒】本週所有課程請務必準備好：" + "、".join(reminders_list) + "。"
    else:
        weekly_reminder_text = "【本週重點行前提醒】本週無特別指定需攜帶或閱讀之項目。"

    return daily_schedule_text, weekly_reminder_text

# --- Gradio 介面設定 ---
with gr.Blocks(title="Google Sheet 課表查詢器") as demo:
    gr.Markdown(
        """
        # 📚 Google Sheet 課表查詢器
        請選擇您想查詢的星期，系統將顯示當日課程並彙整本週提醒。
        """
    )

    # 初始載入資料
    load_schedule_data(SPREADSHEET_ID, SHEET_GID)

    with gr.Row():
        day_input = gr.Dropdown(
            choices=DAY_OPTIONS,
            label="選擇要查詢的星期",
            value=DAY_OPTIONS[2], # 預設為星期三
            interactive=True
        )

    with gr.Column(variant="panel"):
        gr.Markdown("## 🗓 當日課程安排")
        daily_output = gr.Textbox(
            label="當日課程與地點",
            lines=5,
            show_copy_button=True,
            max_lines=5
        )

    with gr.Column(variant="panel"):
        gr.Markdown("## 💡 本週行前提醒 (AI 總結)")
        summary_output = gr.Textbox(
            label="本週重點攜帶/閱讀事項總結",
            lines=2,
            show_copy_button=True
        )

    # 綁定事件
    day_input.change(
        fn=get_schedule_and_summary,
        inputs=day_input,
        outputs=[daily_output, summary_output]
    )

    # 初始載入時執行一次，顯示預設星期的結果
    demo.load(
        fn=lambda day: get_schedule_and_summary(day),
        inputs=day_input,
        outputs=[daily_output, summary_output]
    )


# 執行 Gradio 應用程式
if __name__ == "__main__":
    print("\n程式啟動中... 請稍候，Gradio 介面將在瀏覽器中開啟。")
    try:
        demo.launch(inbrowser=True, share=False)

    except Exception as e:
        print(f"\n執行 Gradio 發生嚴重錯誤: {e}")

🚨 HTTP 錯誤 (請檢查 Sheet ID/GID 或分享設定): 404 Client Error: Not Found for url: https://docs.google.com/spreadsheets/d/https://docs.google.com/spreadsheets/d/1NkL_6hWQZtNkLjTO-YBGgEewK6p0kynV0RFgtVXgF3U/edit?gid=999624801#gid=999624801/export?format=csv&gid=999624801

程式啟動中... 請稍候，Gradio 介面將在瀏覽器中開啟。
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>